In [1]:
from pathlib import Path
import yaml
import os

root_dir = Path("/media") / "sharedData" / "data"
root_name = "glass-a2744"

catalogue_dir = root_dir / "2024_08_16_A2744_v4" / "glass_niriss" / "match_catalogues"

grizli_home_dir = root_dir / "2024_08_16_A2744_v4" / "grizli_home"
grizli_extraction_dir = grizli_home_dir / "Extractions_v2"
os.chdir(grizli_extraction_dir)

import matplotlib as mpl
aanda_columnwidth = 256.0748 / 72.27
aanda_textwidth = 523.5307 / 72.27
rc_fonts = {
    "font.family": "serif",
    "font.size": 9,
    "figure.figsize": (aanda_columnwidth, 3),
    "text.usetex": True,
    "ytick.right": True,
    "ytick.direction": "in",
    "ytick.minor.visible": True,
    "ytick.labelsize": 8,
    "xtick.top": True,
    "xtick.direction": "in",
    "xtick.minor.visible": True,
    "xtick.labelsize": 8,
    "axes.labelsize": 9,
    "axes.titlesize": 9,
    "legend.fontsize": 9,
    "text.latex.preamble": (
        r"""
    \usepackage{txfonts}
    """
    ),
}
mpl.rcParams.update(rc_fonts)

In [2]:
from grizli import jwst_utils, multifit
import logging
import shutil

jwst_utils.QUIET_LEVEL = logging.WARNING
jwst_utils.set_quiet_logging(jwst_utils.QUIET_LEVEL)

In [3]:
from astropy.table import Table, join, Column
import numpy as np

cat_v1 = Table.read(catalogue_dir / "classification_v1" / "compiled_catalogue_v1.fits")

In [4]:


# phot_cat = Table.read(grizli_home_dir / "Extractions" / "glass-a2744_phot.fits")

# from glass_niriss.pipeline import regen_catalogue
# from glass_niriss.sed import temp_chdir
# from astropy.io import fits

# seg_map = fits.getdata(grizli_home_dir / "Prep" / "glass-a2744-ir_seg.fits")
# # print (seg_map.is_file())
# # seg_map.byteswap().newbyteorder()
# # seg_map = seg_map.view(seg_map.dtype.newbyteorder()).astype(np.int32)
# seg_map = np.asarray(seg_map).astype(np.int32)
# #
# import drizzlepac
# print (drizzlepac.__version__)
# import photutils
# print (photutils.__version__)
# import numpy
# print (numpy.__version__)

# with temp_chdir(grizli_home_dir / "Prep"):
#     regen_catalogue(
#         new_seg_map=seg_map,
#         root="glass-a2744-ir",
#         # sci=str(grizli_home_dir / "Prep" / "glass-a2744-f200wn-clear_drc_sci.fits"),
#         # wht=str(grizli_home_dir / "Prep" / "glass-a2744-f200wn-clear_drc_wht.fits"),
#         # sci=str(grizli_home_dir / "Prep" / "glass-a2744-ir_drc_sci.fits"),
#         # wht=str(grizli_home_dir / "Prep" / "glass-a2744-ir_drc_wht.fits"),
#         suffix="test",
#     )

# # # test = join(phot_cat, cat_v1)


In [5]:
from itertools import product
from numpy.lib.recfunctions import structured_to_unstructured

filters = ["F115W","F150W","F200W"]
pas = ["72.0","341.0"]

beam_names = [f"{a},{b}" for b, a in product(pas, filters)]
coverage_names = [f"{b}_COVERAGE" for b in beam_names]
quality_names = [f"{b}_QUALITY" for b in beam_names]
new_names = [f"{b}_USE" for b in beam_names]

v2_cat = cat_v1.copy()

v2_cat.remove_columns(quality_names+new_names)

In [6]:
stage_1_BV = Table.read(grizli_home_dir / "classification-stage-2"/"catalogues"/"output" /"stage_1_Benedetta.fits")
compiled_v1 = Table.read(grizli_home_dir / "classification-stage-2"/"catalogues"/"output" /"compiled_catalogue_v1.fits")
# print (len(stage_1_BV))

# misplaced_ids = np.array([1,8,10,17,52,63,77,99,104,115,120,174])

# stage_1_BV = stage_1_BV[
#     ~(
#         (stage_1_BV["GRIZLI_REDSHIFT"]>=5)
#         & (stage_1_BV["ESTIMATED_REDSHIFT"]==stage_1_BV["GRIZLI_REDSHIFT"])
#     )
# ]
# print (len(stage_1_BV))
# stage_1_BV = stage_1_BV[
#     # ~(
#     #     (stage_1_BV["GRIZLI_REDSHIFT"]>=5)
#     #     & (stage_1_BV["ESTIMATED_REDSHIFT"]==stage_1_BV["GRIZLI_REDSHIFT"])
#     # )
#     ~((stage_1_BV["ID"].astype(int)<260) & (~np.isin(stage_1_BV["ID"].astype(int), misplaced_ids))) 
# ]
# print (len(stage_1_BV))
# stage_1_BV = stage_1_BV[
#     ~(stage_1_BV["UNRELIABLE_REDSHIFT"] | stage_1_BV["BAD_SEG_MAP"])
#     # & ()
#     # # ~(
#     # #     (stage_1_BV["GRIZLI_REDSHIFT"]>=5)
#     # #     & (stage_1_BV["ESTIMATED_REDSHIFT"]==stage_1_BV["GRIZLI_REDSHIFT"])
#     # # )
#     # ~((stage_1_BV["ID"].astype(int)<260) & (~np.isin(stage_1_BV["ID"].astype(int), misplaced_ids))) 
# ]
# print (len(stage_1_BV))
# # print (stage_1_BV)
# v1_idxs = np.isin(compiled_v1["NUMBER"],stage_1_BV["ID"])
# stage_1_BV = stage_1_BV[
#     compiled_v1[v1_idxs]["V1_CLASS"]>=4
# ]
# print (len(stage_1_BV))

lowz_bool = (compiled_v1["V1_CLASS"] >= 4) & (
    ((compiled_v1["REDSHIFT_USE"] < 6) & (compiled_v1["UNRELIABLE_REDSHIFT"] == False))
    | (compiled_v1["zphot"].mask)
)

stage_1_BV = stage_1_BV[np.isin(stage_1_BV["ID"],compiled_v1["NUMBER"][lowz_bool])]

print (len(stage_1_BV))

stage_1_BV.write(grizli_home_dir / "classification-stage-2"/"catalogues"/"output" /"stage_2_Benedetta.fits", overwrite=True)

1024


In [7]:
from numpy.typing import ArrayLike

def table_to_array(table: Table) -> ArrayLike:
    """
    Convert an astropy table to a numpy array.

    Parameters
    ----------
    table : Table
        Original astropy table.

    Returns
    -------
    ArrayLike
        Numpy array.
    """
    return np.lib.recfunctions.structured_to_unstructured(table.as_array()) 

In [8]:
v2_out_dir = grizli_home_dir / "classification-stage-2"/"catalogues" / "compiled"
v2_out_dir.mkdir(exist_ok=True, parents=True)

new_cat_name = "internal_full_7.fits"

if not (v2_out_dir / new_cat_name / "test").is_file():
    stage_2_cats = grizli_home_dir.glob(
        "classification-stage-2/catalogues/output/stage_2*.fits"
    )
    p_names = []
    hist_list = []

    beam_quality_to_int = {"Unusable": 0, "Poor": 1, "Good": 2, "Excellent": 3}

    for c in stage_2_cats:
        p_cat = Table.read(c)
        p_name = c.stem.split("_")[-1]
        p_names.append(p_name)
        print(p_name)
        print (len(p_cat))
        p_cat.sort("SEG_ID")

        _, use_idxs,_ = np.intersect1d(np.array(v2_cat["SEG_ID"]), np.array(p_cat["SEG_ID"]), return_indices=True)

        # print (use_idxs.shape,use_idxs.sum())

        v2_cat["GRIZLI_REDSHIFT"][use_idxs] = p_cat["GRIZLI_REDSHIFT"]

        for n in coverage_names:
            v2_cat[n][use_idxs] = p_cat[n]

        for n in quality_names:
            v2_cat[f"{n}_{p_name}"] = np.nan
            v2_cat[f"{n}_{p_name}"][use_idxs] = [beam_quality_to_int[k] for k in p_cat[n]]

        comment_col = Column(name=f"COMMENTS_{p_name}", dtype=p_cat["COMMENTS"].dtype, length=len(v2_cat))
        comment_col[use_idxs] = p_cat["COMMENTS"]
        v2_cat.add_column(comment_col)

        v2_cat[f"ESTIMATED_REDSHIFT_{p_name}"] = np.nan
        v2_cat[f"ESTIMATED_REDSHIFT_{p_name}"][use_idxs] = p_cat["ESTIMATED_REDSHIFT"]

        v2_cat[f"Z_FLAG_{p_name}"] = np.nan
        flag = np.full(len(p_cat["ESTIMATED_REDSHIFT"]), np.nan)
        flag[np.isfinite(p_cat["ESTIMATED_REDSHIFT"])] = 2
        flag[p_cat["TENTATIVE_REDSHIFT"]] = 1
        flag[p_cat["UNRELIABLE_REDSHIFT"]] = 0
        v2_cat[f"Z_FLAG_{p_name}"][use_idxs] = flag
        # if p_name=="Peter":
        #     # print (p_cat["TENTATIVE_REDSHIFT"])
        #     # print (np.unique(np.array(p_cat["TENTATIVE_REDSHIFT"]), return_counts=True))
        #     # print (np.unique(np.array(p_cat["UNRELIABLE_REDSHIFT"]), return_counts=True))
        #     # print (np.unique(v2_cat["Z_FLAG_Peter"], return_counts=True))
        #     print (p_cat[75:85].pprint_all())
        #     print (flag[75:85])
        #     print (v2_cat["SEG_ID", f"Z_FLAG_{p_name}"][use_idxs][75:85])
        #     exit()
        # print (len(p_cat["UNRELIABLE_REDSHIFT"]))
        # v2_cat[f"Z_FLAG_{p_name}"][use_idxs] = [0 if k for k in p_cat["UNRELIABLE_REDSHIFT"]>0 if k]
        # v2_cat[f"Z_FLAG_{p_name}"][use_idxs][p_cat["UNRELIABLE_REDSHIFT"]] = 0
        # v2_cat[f"Z_FLAG_{p_name}"][use_idxs][p_cat["TENTATIVE_REDSHIFT"]] = 1
        # print (v2_cat[f"Z_FLAG_{p_name}"])

        hist_list.append(p_cat["ESTIMATED_REDSHIFT"][v2_cat[use_idxs][f"Z_FLAG_{p_name}"]==2])

    # from pprint import pprint

    z_val_names = [z for z in v2_cat.colnames if "ESTIMATED_REDSHIFT_" in z]
    z_flag_names = [z for z in v2_cat.colnames if "Z_FLAG_" in z]


    # print(z_flag_names)

    # print(v2_cat.colnames)

    # v2_cat["Z_VALS_STR"] = np.n
    # z_vals_list = np.empty(len(v2_cat),dtype=float)
    # delim_arr = np.full_like(z_vals_str, ";", dtype=str)
    # from numpy.char import add
    # for n in z_val_names:
    #     # z_vals
    #     print(v2_cat[n].astype("|S6")) 
    #     z_vals_str = add(z_vals_str, add(delim_arr,v2_cat[n].astype("|S6")))
    #     print (z_vals_str)
    # print (table_to_array(v2_cat[z_val_names]))
    v2_cat["Z_VALS"] = table_to_array(v2_cat[z_val_names])
    # v2_cat["Z_FLAGS"] = table_to_array(v2_cat[z_flag_names]).astype(int).astype(str)
    
    from numpy import ma
    flags = table_to_array(v2_cat[z_flag_names])
    flags_masked = ma.masked_invalid(flags).astype(int).astype(str)

    print (flags_masked[100])

    v2_cat["Z_FLAGS"] = flags_masked
    print (v2_cat["Z_FLAGS"][100].dtype)
    print (v2_cat["Z_FLAGS"].dtype)

    # exit()
    # print (flags[100])
    # print (flags_masked[100])
    # # print (table_to_array(v2_cat[z_flag_names])[100])
    # # print (table_to_array(v2_cat[z_flag_names])[100].astype(int))
    # # print (table_to_array(v2_cat[z_flag_names])[100].astype(int).astype(str))
    # # print (table_to_array(v2_cat[z_flag_names])[100].astype(str))

    # exit()

    # print (v2_cat["Z_VALS"][100])
    # print (v2_cat["Z_FLAGS"][100])

    v2_cat["Z_FLAG_ALL"] = -1000
    v2_cat["Z_EST_ALL"] = np.nan
    v2_cat["N_CLASS"] = 0

    Z_FLAG_DESCRIPTION = {
        "secure>=2, range(z_est)<=1%": 10,
        "secure>=2, range(z_est)>1%;<=5%": 9,
        "secure>=2, range(z_est)>5%": 8,
        # "All tentative, >=2 views, estimates consistent within 1%, agrees with zphot/zspec within 5%": 7,
        "tentative>=2, range(z_est)<=1%": 7,
        "tentative>=2, range(z_est)>1%;<=5%": 6,
        "tentative>=2, range(z_est)>5%": 5,
        "unreliable>=2": -10,
        "secure>=1, tentative==1, range(z_est[secure])<=5%": 4,
        "secure>=1, tentative==1, range(z_est[secure])>5%": 3,
        "tentative>1, secure==1, range(z_est[secure])<=5%": 2,
        "tentative>1, secure==1, range(z_est[secure])>5%": 1,
        "secure>=1, unreliable==1": 0,
        "tentative>=1, unreliable==1": -1,
        "unreliable>=1, secure|tentative>=1": -2,
        "n<2": -33,
        "UNCLASSIFIED": -99,
        "ERROR": -1000,
        # "All tentative, >=2 views, estimates consistent within 1%": 6,
        # "All tentative, >=2 views, estimates consistent within 1-5%": 5,
        # "All tentative, >=2 views, estimates inconsistent within 5%": 4,
    }

    Z_FLAG_INV = {v: k for k, v in Z_FLAG_DESCRIPTION.items()}
    print(Z_FLAG_INV)

    for i, row in enumerate(v2_cat[:]):
        if row["V1_CLASS"] <= 3:
            v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION["UNCLASSIFIED"]
            continue
        # np.isfinite?
        z_vals = structured_to_unstructured(np.array(row[z_val_names]))
        z_flags = structured_to_unstructured(np.array(row[z_flag_names]))

        fin = np.isfinite(z_flags)
        v2_cat["N_CLASS"][i] = np.sum(fin)
        # print (fin)
        if np.sum(fin) >= 2:
            if np.all(z_flags[fin] == 2):
                # print ("y")
                # print (f"ID={row["ID"]}, z={z_vals[fin]}")
                # print (np.ptp(z_vals[fin]), 0.01*np.nanmedian(z_vals))
                if np.ptp(z_vals[fin]) <= 0.01 * np.nanmedian(z_vals):
                    v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                    v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION[
                        "secure>=2, range(z_est)<=1%"
                    ]
                elif np.ptp(z_vals[fin]) <= 0.05 * np.nanmedian(z_vals):
                    v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                    v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION[
                        "secure>=2, range(z_est)>1%;<=5%"
                    ]
                else:
                    v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                    v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION[
                        "secure>=2, range(z_est)>5%"
                    ]

            elif np.all(z_flags[fin] == 1):
                # print ("y")
                # print (f"ID={row["ID"]}, z={z_vals[fin]}")
                # print (np.ptp(z_vals[fin]), 0.01*np.nanmedian(z_vals))
                # if (np.ptp(z_vals[fin]) <= 0.01 * np.nanmedian(z_vals)) and (
                #     np.abs(np.nanmedian(z_vals) - row["zphot"]) <= 0.01 * row["zphot"]
                # ):
                #     v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                #     v2_cat["Z_FLAG_ALL"][i] = 7
                if np.ptp(z_vals[fin]) <= 0.01 * np.nanmedian(z_vals):
                    v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                    v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION[
                        "tentative>=2, range(z_est)<=1%"
                    ]
                elif np.ptp(z_vals[fin]) <= 0.05 * np.nanmedian(z_vals):
                    v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                    v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION[
                        "tentative>=2, range(z_est)>1%;<=5%"
                    ]
                else:
                    v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                    v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION[
                        "tentative>=2, range(z_est)>5%"
                    ]

            elif (np.sum(z_flags[fin] == 1) == 1) and (np.sum(z_flags[fin] == 2) >= 1):
                if np.ptp(z_vals[fin]) <= 0.05 * np.nanmedian(z_vals):
                    v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                    v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION[
                        "secure>=1, tentative==1, range(z_est[secure])<=5%"
                    ]
                else:
                    v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                    v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION[
                        "secure>=1, tentative==1, range(z_est[secure])>5%"
                    ]

            elif (np.sum(z_flags[fin] == 1) > 1) and (np.sum(z_flags[fin] == 2) == 1):
                if np.ptp(z_vals[fin]) <= 0.05 * np.nanmedian(z_vals):
                    v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                    v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION[
                        "tentative>1, secure==1, range(z_est[secure])<=5%"
                    ]
                else:
                    v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                    v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION[
                        "tentative>1, secure==1, range(z_est[secure])>5%"
                    ]

            elif (np.sum(z_flags[fin] == 2) >= 1) and (np.sum(z_flags[fin] == 0) == 1):
                v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION["secure>=1, unreliable==1"]

            elif (np.sum(z_flags[fin] == 1) >= 1) and (np.sum(z_flags[fin] == 0) == 1):
                v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION["tentative>=1, unreliable==1"]

            elif np.all(z_flags[fin] == 0):
                v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION["unreliable>=2"]

            elif np.sum(z_flags[fin] == 0) >= 1:
                v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
                v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION[
                    "unreliable>=1, secure|tentative>=1"
                ]

            else:
                pass
        else:
            v2_cat["Z_EST_ALL"][i] = np.nanmedian(z_vals[fin])
            v2_cat["Z_FLAG_ALL"][i] = Z_FLAG_DESCRIPTION["n<2"]

    # print (f"ID={row["ID"]}, z={np.nanmedian(z_vals):.3f}, flags={np.nanmean(z_flags)}")
    # with np.printoptions(precision=3, suppress=True,floatmode='fixed'):
    #     print (z_vals)
    #     print (z_flags)
    # # print (row[z_val_names])
    # # print (row[z_flag_names])
    v2_cat.write(v2_out_dir / new_cat_name, overwrite=True)
else:
    v2_cat = Table.read(v2_out_dir / new_cat_name)


Benedetta
1024
Guido
560
Matt
220
Nicolo
450
Peter
582
Sofia
559
Tommaso
184
Xianlong
450
Yechi
550
['2' -- -- -- -- -- -- '1' --]
<U21
<U21
{10: 'secure>=2, range(z_est)<=1%', 9: 'secure>=2, range(z_est)>1%;<=5%', 8: 'secure>=2, range(z_est)>5%', 7: 'tentative>=2, range(z_est)<=1%', 6: 'tentative>=2, range(z_est)>1%;<=5%', 5: 'tentative>=2, range(z_est)>5%', -10: 'unreliable>=2', 4: 'secure>=1, tentative==1, range(z_est[secure])<=5%', 3: 'secure>=1, tentative==1, range(z_est[secure])>5%', 2: 'tentative>1, secure==1, range(z_est[secure])<=5%', 1: 'tentative>1, secure==1, range(z_est[secure])>5%', 0: 'secure>=1, unreliable==1', -1: 'tentative>=1, unreliable==1', -2: 'unreliable>=1, secure|tentative>=1', -33: 'n<2', -99: 'UNCLASSIFIED', -1000: 'ERROR'}


In [9]:
print("\nGalaxies classified in Stage 2:\n")

n_class = np.unique(
    np.array(v2_cat["N_CLASS"][v2_cat["V1_CLASS"] >= 4]), return_counts=True
)
for v, c in zip(*n_class):
    print(f"#Classifications = {v:<5} Counts = {c}")

print("")

flag_types = np.unique(
    np.array(v2_cat["Z_FLAG_ALL"][v2_cat["V1_CLASS"] >= 4]), return_counts=True
)
for v, c in zip(*flag_types):
    print(f"Flag = {v:<5} Counts = {c:<5} Description: {Z_FLAG_INV[v]}")

print("\nlow_z_cat:")
n_class = np.unique(
    np.array(
        v2_cat["N_CLASS"][
            (v2_cat["V1_CLASS"] >= 4)
            & (
                (
                    (v2_cat["REDSHIFT_USE"] < 6)
                    & (v2_cat["UNRELIABLE_REDSHIFT"] == False)
                )
                | (~np.isfinite(v2_cat["zphot"]))
            )
        ]
    ),
    return_counts=True,
)
for v, c in zip(*n_class):
    print(f"#Classifications = {v:<5} Counts = {c}")

print("\nhigh_z_cat:")
n_class = np.unique(
    np.array(
        v2_cat["N_CLASS"][
            (v2_cat["V1_CLASS"] >= 4)
            & (
                (
                    (v2_cat["REDSHIFT_USE"] >= 6)
                    | (v2_cat["UNRELIABLE_REDSHIFT"] == True)
                )
                & (np.isfinite(v2_cat["zphot"]))
            )
        ]
    ),
    return_counts=True,
)
for v, c in zip(*n_class):
    print(f"#Classifications = {v:<5} Counts = {c}")

print(
    np.intersect1d(
        np.array(
            v2_cat["ID"][
                (v2_cat["V1_CLASS"] >= 4)
                & (
                    (
                        (v2_cat["REDSHIFT_USE"] >= 6)
                        | (v2_cat["UNRELIABLE_REDSHIFT"] == True)
                    )
                    & (np.isfinite(v2_cat["zphot"]))
                )
            ]
        ),
        np.array(
            v2_cat["ID"][
                (v2_cat["V1_CLASS"] >= 4)
                & (
                    (
                        (v2_cat["REDSHIFT_USE"] < 6)
                        & (v2_cat["UNRELIABLE_REDSHIFT"] == False)
                    )
                    | (~np.isfinite(v2_cat["zphot"]))
                )
            ]
        ),
    )
)

print(
    len(
        v2_cat["ID"][
            ((v2_cat["Z_FLAG_ALL"] == Z_FLAG_DESCRIPTION["secure>=2, range(z_est)<=1%"]) | (v2_cat["Z_FLAG_ALL"] == Z_FLAG_DESCRIPTION["secure>=2, range(z_est)>1%;<=5%"]))
            & (~np.isfinite(v2_cat["zspec"]))
        ]
    )
)


Galaxies classified in Stage 2:

#Classifications = 2     Counts = 1541
#Classifications = 3     Counts = 499

Flag = -10   Counts = 410   Description: unreliable>=2
Flag = -2    Counts = 60    Description: unreliable>=1, secure|tentative>=1
Flag = -1    Counts = 300   Description: tentative>=1, unreliable==1
Flag = 0     Counts = 456   Description: secure>=1, unreliable==1
Flag = 1     Counts = 5     Description: tentative>1, secure==1, range(z_est[secure])>5%
Flag = 2     Counts = 16    Description: tentative>1, secure==1, range(z_est[secure])<=5%
Flag = 3     Counts = 104   Description: secure>=1, tentative==1, range(z_est[secure])>5%
Flag = 4     Counts = 194   Description: secure>=1, tentative==1, range(z_est[secure])<=5%
Flag = 5     Counts = 7     Description: tentative>=2, range(z_est)>5%
Flag = 6     Counts = 2     Description: tentative>=2, range(z_est)>1%;<=5%
Flag = 7     Counts = 46    Description: tentative>=2, range(z_est)<=1%
Flag = 8     Counts = 39    Description: se